## Architecture

    P -> Encoder-lstm -> match-lstm -> PTR-Net
    Q -> Encoder-lstm -^ 

In [30]:
import os
import re

import numpy as np

from numpy import array
from keras.models import Sequential , Model
from keras.layers import Dense ,Input ,Lambda 
from keras.layers import TimeDistributed
from keras.layers import LSTM , Embedding
from keras.layers import merge,Dot,Multiply,Reshape,concatenate,RepeatVector

#from keras.backend import softmax #無法設定axis
from keras.activations import softmax #可設定axis

In [69]:
hidden_layer_p = 128
hidden_layer_q = 128



max_review_length = 5000
question_length = 20

input_p = Input((max_review_length,),name='paragraph')
input_q = Input((question_length,),name='question')

# input_dim is len(vocab) like the account of word. 
embd = Embedding(input_dim=100,output_dim=128)

embd_p = embd(input_p)
embd_q = embd(input_q)

encoder_p = LSTM(hidden_layer_p,return_sequences=True,name='paragraph_encoder')
encoder_q = LSTM(hidden_layer_q,name='question_encoder')
enco_p = encoder_p(embd_p)
enco_q = encoder_q(embd_q)

enco_q = RepeatVector(max_review_length)(enco_q)

attn = concatenate([enco_p,enco_q],name='QP_concat')

attn = TimeDistributed(Dense(5,activation='tanh'))(attn)
attn = TimeDistributed(Dense(1,activation='tanh'))(attn)
attn = Lambda(lambda x: softmax(x,axis=-2),name='attention_by_softmax')(attn)

#attn_vector = Multiply()([attn,enco_p])

## decoder
comprehension_p = LSTM(hidden_layer_p,go_backwards=True,return_sequences=True,name='compre_encoder')(embd_p)

attn_comprehension_p = Multiply(name='attn_on_compre_vector')([attn,comprehension_p])

deco_p = LSTM(hidden_layer_p,return_sequences=True,name='decoder_p')(attn_comprehension_p)

ptr_attn = TimeDistributed(Dense(1,activation='tanh'))(deco_p)
ptr_attn_value = Lambda(lambda x: softmax(x,axis=-2),name='point_layer')(ptr_attn)

In [70]:
model = Model(inputs=[input_p,input_q],outputs=[ptr_attn_value])
model.compile(loss='cosine_proximity',optimizer='adam') #cosine_proximity
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
paragraph (InputLayer)           (None, 5000)          0                                            
____________________________________________________________________________________________________
question (InputLayer)            (None, 20)            0                                            
____________________________________________________________________________________________________
embedding_27 (Embedding)         multiple              12800       paragraph[0][0]                  
                                                                   question[0][0]                   
____________________________________________________________________________________________________
question_encoder (LSTM)          (None, 128)           131584      embedding_27[1][0]      

In [71]:
from keras.utils import plot_model
plot_model(model , to_file='QASystem.png')